In [1]:
"""
@Name: tensorboard_visualization.py
@Auth: cc980
@Date: 2022/9/25-14:30
@Desc: Tensorboard的可视化
@Ver : 1.0.0
"""
import keras
import tensorflow as tf
import datetime
import os

print("Tensorflow version: {}".format(tf.__version__))
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()
print(train_image.shape)
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)

Tensorflow version: 2.10.0
(60000, 28, 28)


In [2]:
train_image = tf.cast(train_image / 255, tf.float32)
test_image = tf.cast(test_image / 255, tf.float32)
train_label = tf.cast(train_label, tf.int64)
test_label = tf.cast(test_label, tf.int64)

In [3]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_image, train_label))
dataset_test = tf.data.Dataset.from_tensor_slices((test_image, test_label))

In [4]:
dataset_train = dataset_train.shuffle(10000).repeat().batch(64)
dataset_test = dataset_test.repeat().batch(64)

In [5]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], input_shape=(28, 28, 1), activation="relu"),
    tf.keras.layers.Conv2D(32, [3, 3], activation="relu"),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 global_max_pooling2d (Globa  (None, 32)               0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["acc"]
)

In [8]:
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

In [9]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=60000//64,
    validation_data=dataset_test,
    validation_steps=10000//64,
    callbacks=[tensorboard_callback]
)

Epoch 1/5
937/937 [==============================] - 62s 64ms/step - loss: 1.0482 - acc: 0.6930 - val_loss: 0.4548 - val_acc: 0.8641
Epoch 2/5
937/937 [==============================] - 58s 62ms/step - loss: 0.3948 - acc: 0.8767 - val_loss: 0.3453 - val_acc: 0.8923
Epoch 3/5
937/937 [==============================] - 57s 61ms/step - loss: 0.3247 - acc: 0.8974 - val_loss: 0.2795 - val_acc: 0.9126
Epoch 4/5
937/937 [==============================] - 57s 61ms/step - loss: 0.2888 - acc: 0.9080 - val_loss: 0.2570 - val_acc: 0.9209
Epoch 5/5
937/937 [==============================] - 60s 64ms/step - loss: 0.2656 - acc: 0.9155 - val_loss: 0.2487 - val_acc: 0.9230


### 记录自定义标量
重新调整回归模型并记录自定义学习率。这是如何做：
使用创建文件编写器tf.summary.create_file_writer().
定义自定义学习率功能，这将传递给Keras LearningRateScheduler回调。
在学习率功能内，用户tf.summary.scalar()记录自定义学习率。
将LearningRateScheduler回调传递给Model.fit()。
通常，要记录自定义标量，您需要使用tf.summary.scalar()文件编写器。文件编写器负责将此运行的数据写入指定的目录，并在使用时隐式使用tf.summary.scalar()
